In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from glob import glob
import os
import json 
import random

In [4]:
path = os.getcwd()

In [5]:
# 분석에 사용할 feature 선택
csv_features = ['내부 온도 1 평균', '내부 온도 1 최고', '내부 온도 1 최저', '내부 습도 1 평균', '내부 습도 1 최고', 
                '내부 습도 1 최저', '내부 이슬점 평균', '내부 이슬점 최고', '내부 이슬점 최저']

# csv_files = sorted(glob('sample_data/*/*.csv'))
csv_files = sorted(glob(path + '\\sample_data\\sample_data\\*\\*.csv'))

temp_csv = pd.read_csv(csv_files[0])[csv_features]
max_arr, min_arr = temp_csv.max().to_numpy(), temp_csv.min().to_numpy()

# feature 별 최대값, 최솟값 계산
for csv in tqdm(csv_files[1:]):
    temp_csv = pd.read_csv(csv)[csv_features]
    temp_max, temp_min = temp_csv.max().to_numpy(), temp_csv.min().to_numpy()
    max_arr = np.max([max_arr,temp_max], axis=0)
    min_arr = np.min([min_arr,temp_min], axis=0)

# feature 별 최대값, 최솟값 dictionary 생성
csv_feature_dict = {csv_features[i]:[min_arr[i], max_arr[i]] for i in range(len(csv_features))}
csv_feature_dict

100%|██████████| 498/498 [00:02<00:00, 244.05it/s]


{'내부 온도 1 평균': [14.4, 47.3],
 '내부 온도 1 최고': [14.5, 47.6],
 '내부 온도 1 최저': [14.4, 47.0],
 '내부 습도 1 평균': [34.1, 100.0],
 '내부 습도 1 최고': [36.5, 100.0],
 '내부 습도 1 최저': [32.4, 100.0],
 '내부 이슬점 평균': [12.4, 29.9],
 '내부 이슬점 최고': [12.8, 31.9],
 '내부 이슬점 최저': [12.1, 29.1]}

In [6]:
# 제공된 sample data는 파프리카와 시설포도 2종류의 작물만 존재
label_description = {
 '3_00_0': '파프리카_정상',
 '3_a9_1': '파프리카흰가루병_초기',
 '3_a9_2': '파프리카흰가루병_중기',
 '3_a9_3': '파프리카흰가루병_말기',
 '3_a10_1': '파프리카잘록병_초기',
 '3_a10_2': '파프리카잘록병_중기',
 '3_a10_3': '파프리카잘록병_말기',
 '3_b3_1': '칼슘결핍_초기',
 '3_b3_2': '칼슘결핍_중기',
 '3_b3_3': '칼슘결핍_말기',
 '3_b6_1': '다량원소결핍 (N)_초기',
 '3_b6_2': '다량원소결핍 (N)_중기',
 '3_b6_3': '다량원소결핍 (N)_말기',
 '3_b7_1': '다량원소결핍 (P)_초기',
 '3_b7_2': '다량원소결핍 (P)_중기',
 '3_b7_3': '다량원소결핍 (P)_말기',
 '3_b8_1': '다량원소결핍 (K)_초기',
 '3_b8_2': '다량원소결핍 (K)_중기',
 '3_b8_3': '다량원소결핍 (K)_말기',
 '6_00_0': '시설포도_정상',
 '6_a11_1': '시설포도탄저병_초기',
 '6_a11_2': '시설포도탄저병_중기',
 '6_a11_3': '시설포도탄저병_말기',
 '6_a12_1': '시설포도노균병_초기',
 '6_a12_2': '시설포도노균병_중기',
 '6_a12_3': '시설포도노균병_말기',
 '6_b4_1': '일소피해_초기',
 '6_b4_2': '일소피해_중기',
 '6_b4_3': '일소피해_말기',
 '6_b5_1': '축과병_초기',
 '6_b5_2': '축과병_중기',
 '6_b5_3': '축과병_말기',
}

label_encoder = {key:idx for idx, key in enumerate(label_description)}
label_decoder = {val:key for key, val in label_encoder.items()}

In [7]:
data_files = glob(path + '\\sample_data\\sample_data\\*')
csvlen = []
for i in data_files:
    
    if glob(i + '\\*.csv') == []:   #10462폴더 비어있음. 다음에 확인해보기
        pass
    else:
        df = pd.read_csv(glob(i + '\\*.csv')[0])
        csvlen.append(len(df))
    
print(csvlen)



[291, 588, 294, 294, 294, 294, 588, 588, 588, 291, 294, 294, 294, 294, 294, 588, 294, 294, 588, 588, 294, 294, 588, 291, 588, 294, 588, 291, 294, 294, 588, 294, 294, 294, 588, 294, 294, 294, 294, 588, 291, 294, 294, 294, 294, 294, 294, 294, 294, 588, 588, 294, 294, 588, 294, 588, 588, 294, 294, 294, 588, 294, 294, 588, 294, 291, 588, 291, 294, 294, 588, 294, 588, 294, 294, 294, 291, 291, 588, 294, 294, 588, 262, 291, 291, 294, 294, 291, 291, 294, 294, 294, 294, 588, 294, 294, 588, 588, 294, 294, 293, 588, 291, 294, 588, 294, 294, 588, 294, 294, 291, 294, 294, 294, 294, 294, 588, 294, 294, 294, 294, 294, 588, 291, 294, 588, 294, 294, 588, 588, 294, 294, 588, 588, 294, 294, 291, 294, 588, 588, 291, 294, 294, 294, 294, 294, 294, 588, 294, 588, 588, 294, 294, 588, 294, 294, 294, 294, 294, 294, 291, 294, 294, 294, 294, 294, 291, 588, 294, 294, 588, 291, 294, 294, 294, 294, 294, 294, 294, 294, 588, 291, 291, 291, 294, 294, 291, 294, 294, 294, 294, 291, 588, 294, 294, 588, 588, 294, 294, 294,

In [8]:
min(csvlen)

262

In [9]:
class DataController():
    def __init__(self,csvfeatures,csvfeaturedict):
        self.csv_features = csvfeatures
        self.csv_feature_dict = csvfeaturedict
    
    def road_csv(self,foldnam,timenum):
        df = pd.read_csv(foldnam)
        return df.loc[:timenum-1,self.csv_features]      #csv파일 제일 짧은게 291개임 오류인지는 모르겠으나 일단 최소길이로 통일하여 처리
    
    def scaling(self,minmaxdic,df):
        for col in minmaxdic.keys():
            df.loc[:,col] = df.loc[:,col] - csv_feature_dict[col][0]
            df.loc[:,col] = df.loc[:,col] / (csv_feature_dict[col][1]-csv_feature_dict[col][0])
        return df
    
    
    def getimage(self,imgpath):
        img = cv2.imread(imgpath)
        img = cv2.resize(img, dsize=(256, 256), interpolation=cv2.INTER_AREA)
        img = img.astype(np.float32)/255  ##픽셀값을 0~1사이로 정규화
        # img = np.transpose(img, (2,0,1))
        return img
    
    def getlable(self,jsonpath):
        with open(jsonpath, 'r') as f:
            json_file = json.load(f)

        crop = json_file['annotations']['crop']
        disease = json_file['annotations']['disease']
        risk = json_file['annotations']['risk']
        label = f'{crop}_{disease}_{risk}'
        return label
    
    def getdata(self,datapath,timenum,featnum):

        csvarr = np.empty((0,timenum,featnum), float)
        imgarr = np.empty((0,256,256,3), float)
        lablearr = np.array([])
        
        # predictor = np.append(predictor,ndf.reshape(-1,timenum,featnum),axis = 0)
        
        for ind,i in enumerate(datapath):
            
            if glob(i + '\\*.csv') == []:  #10462폴더 비어있음. 다음에 확인해보기
                pass
            else:
                csvpath = glob(i + '\\*.csv')[0]
                imgpath = glob(i + '\\*.jpg')[0]
                jsonpath = glob(i + '\\*.json')[0]
                # con = DataController()
                df = self.road_csv(csvpath,timenum)
                df2 = self.scaling(csv_feature_dict,df).to_numpy().reshape(-1,timenum,featnum)
                imgdata = self.getimage(imgpath).reshape(-1,256,256,3)
                label = label_encoder[self.getlable(jsonpath)]
                # label = self.getlable(jsonpath)
                
                csvarr = np.append(csvarr,df2, axis = 0)
                imgarr = np.append(imgarr,imgdata, axis = 0)
                lablearr = np.append(lablearr,label)
            
        return [csvarr,imgarr],lablearr
            
        
        
        
          

In [10]:
#프로젝트에 있는 모든 데이터폴터 불러오기
data_files = glob(path + '\\sample_data\\sample_data\\*')
#셔플
random.shuffle(data_files)
#앞에서 300번째까지 트레인셋으로
trainfiles = data_files[:400]
#나머지는 테스트셋으로
testfiles = data_files[400:]

# 데이터 컨트롤러
dacon = DataController(csv_features,csv_feature_dict)
# 배치화된 데이터셋 만들기
# test셋용으로는 train = False로 하여 배치안생성하게됨
x_train,y_train = dacon.getdata(trainfiles,260,9)
x_test,y_test = dacon.getdata(testfiles,260,9)


In [11]:
import tensorflow as tf
from tensorflow import keras

In [12]:
class paraBlock(keras.layers.Layer):
    def __init__(self):
        super(paraBlock, self).__init__()
        self.block1_layer1 = tf.keras.layers.LSTM(128)
        self.block1_layer2 = tf.keras.layers.Dense(64)
        self.block1_layer3 = tf.keras.layers.Dense(32)
        self.block1_layer4 = tf.keras.layers.Dense(16)
        
        self.block2_layer1 = tf.keras.layers.Conv2D( filters=16, kernel_size=10, activation='relu')
        self.block2_layer2 = tf.keras.layers.Conv2D( filters=32, kernel_size=10, activation='relu')
        self.block2_layer3 = tf.keras.layers.Conv2D( filters=64, kernel_size=10, activation='relu')
        self.block2_layer4 = tf.keras.layers.Conv2D( filters=128, kernel_size=10, activation='relu')
        
        self.pooling_layer = tf.keras.layers.MaxPooling2D((2, 2))
        self.flat_layer = tf.keras.layers.Flatten()
        
        self.out_layer1 = tf.keras.layers.Dense(64,activation='relu')
        self.out_layer2 = tf.keras.layers.Dense(32,activation='softmax')
        
    def call(self, inputs):
        #LSTM파트
        lstm_x = self.block1_layer1(inputs[0])
        lstm_x = tf.nn.relu(lstm_x)
        lstm_x = self.block1_layer2(lstm_x)
        lstm_x = tf.nn.relu(lstm_x)
        lstm_x = self.block1_layer3(lstm_x)
        lstm_x = tf.nn.relu(lstm_x)
        lstm_x = self.block1_layer4(lstm_x)
        # x = tf.nn.relu(x)
        # x = self.layer5(x)
        
        #CNN파트
        cnn_x = self.block2_layer1(inputs[1])
        cnn_x = self.pooling_layer(cnn_x)
        
        cnn_x = self.block2_layer2(cnn_x)
        cnn_x = self.pooling_layer(cnn_x)
        
        cnn_x = self.block2_layer3(cnn_x)
        cnn_x = self.pooling_layer(cnn_x)
        
        cnn_x = self.block2_layer4(cnn_x)
        cnn_x = self.flat_layer(cnn_x)
        
        #합치기
        out = tf.concat([lstm_x,cnn_x],axis=1)
        out = self.out_layer1(out)
        out = self.out_layer2(out)
        
        return out
        

In [13]:
class Paramodel(keras.Model):
    def __init__(self, name = None):
        super(Paramodel, self).__init__()
    
        self.module1 = paraBlock()
        
    def call(self, inputs):
        x1 = self.module1(inputs)
    
        # x = tf.concat([x1,x2,x3],axis=1)
        
        
        return(x1)
    
# inputs = tf.random.normal([3,2,10,8])
# model1 = Paramodel(name='model_01')
# print(model1(inputs))
# model1.variables,model1.summary()

In [14]:
model1 = Paramodel(name='model_01')
inputs = x_train
print(model1(inputs))

tf.Tensor(
[[0.03534087 0.03173486 0.02937816 ... 0.03081517 0.03039897 0.03134452]
 [0.03605588 0.0313326  0.02957969 ... 0.03046688 0.03148051 0.03194847]
 [0.03503105 0.03170215 0.03005319 ... 0.03042696 0.03162529 0.03117512]
 ...
 [0.03660838 0.03142332 0.02915426 ... 0.03074797 0.03039005 0.03124927]
 [0.03620866 0.03141771 0.02952239 ... 0.03054002 0.03093392 0.0314289 ]
 [0.03609478 0.03178464 0.02955336 ... 0.03031245 0.03136037 0.03164868]], shape=(400, 32), dtype=float32)


In [15]:
# Instantiate an optimizer.
optimizer = keras.optimizers.Adam(learning_rate=1e-3)
# Instantiate a loss function.
loss_fn = keras.losses.SparseCategoricalCrossentropy(from_logits=False)

In [17]:
epochs = 2
for epoch in range(epochs):
    print("\nStart of epoch %d" % (epoch,))

    # Iterate over the batches of the dataset.
    for step in range(1):

        # Open a GradientTape to record the operations run
        # during the forward pass, which enables auto-differentiation.
        with tf.GradientTape() as tape:

            # Run the forward pass of the layer.
            # The operations that the layer applies
            # to its inputs are going to be recorded
            # on the GradientTape.
            logits = model1(x_train, training=True)  # Logits for this minibatch

            # Compute the loss value for this minibatch.
            loss_value = loss_fn(y_train, logits)
            #label_encoder
        # Use the gradient tape to automatically retrieve
        # the gradients of the trainable variables with respect to the loss.
        grads = tape.gradient(loss_value, model1.trainable_weights)

        # Run one step of gradient descent by updating
        # the value of the variables to minimize the loss.
        optimizer.apply_gradients(zip(grads, model1.trainable_weights))

        # Log every 200 batches.
        if step % 200 == 0:
            print(
                "Training loss (for one batch) at step %d: %.4f"
                % (step, float(loss_value))
            )
            print("Seen so far: %s samples" % ((step + 1)))


Start of epoch 0
Training loss (for one batch) at step 0: 39.3254
Seen so far: 1 samples

Start of epoch 1
Training loss (for one batch) at step 0: 2.9413
Seen so far: 1 samples


In [20]:
y_pred = model1(x_test)
score = loss_fn(y_test,y_pred)
print("모델성능: ",score)


모델성능:  tf.Tensor(3.3808637, shape=(), dtype=float32)
